# Interaction with the file system (2)

## interesting alternatives: the `pathlib` and `shutil` modules

These modules work a bit differently than the `os` module. While the `os` module is purely functional and the path is fed in as a string, the `pathlib` transforms the given path into an _object_ which then offers a number of _methods_ related to that object.

**import pathlib and shutil**

In [ ]:
import pathlib
import shutil

### test if a file or path exists

In [ ]:
file = pathlib.Path('/home/ihritik/Desktop/file.txt') 

In [ ]:
file.exists()

In [ ]:
directory = pathlib.Path('.')

In [ ]:
directory.exists()

In [ ]:
directory.is_dir()

In [ ]:
directory.is_symlink()

get the `stat` info from the `os` module above

In [ ]:
stat = directory.stat()

In [ ]:
oct(stat.st_mode & 0o777)

the `absolute()` method returns, not surprisingly, the absolute path. Well, not exactly. It returns a `PosixPath` object:

In [ ]:
directory.absolute()

We can get a normal string representation of it:

In [ ]:
directory.absolute().as_posix()

... or a URI represention:

In [ ]:
directory.absolute().as_uri()

### create a file, test if it exists, delete the file

In [ ]:
file = pathlib.Path('_pathlib_testfile')
file.touch()

In [ ]:
file.exists()

In [ ]:
file.unlink()

### open file, write content, close file

In [ ]:
file = pathlib.Path('_pathlib_file_with_content')

In [ ]:
file.write_text('here comes some content')

In [ ]:
file.read_text()

overwrite content

In [ ]:
file.write_text('some more content')

In [ ]:
file.read_text()

delete the file

In [ ]:
file.unlink()

In [ ]:
file.exists()

### change ownership of a file

In [ ]:
info = pathlib.Path('_pathlib_ownership_testfile')
info.touch()

In [ ]:
print("owner:", info.owner())
print("group:", info.group())

In [ ]:
shutil.chown(path='_pathlib_ownership_testfile', group='everyone')

In [ ]:
info.group()

In [ ]:
info.unlink()

### copy files around

This implements the `cp` command

In [ ]:
import shutil
import os
source = os.listdir(".")
destination = "backup_folder"
if not os.path.exists(destination):
    os.mkdir(destination)
for file in source:
    if file.endswith(".ipynb"):
        shutil.copy(file,destination)

In [ ]:
os.listdir(destination)

### move files

This implements the `mv` command

In [ ]:
!touch requirements.txt

In [ ]:
new_destination = "backup_folder_2"
if not os.path.exists(new_destination):
    os.mkdir(new_destination)
source = os.listdir(".")
for file in source:
    if file.endswith(".txt"):
        print(file)
        shutil.move(file, new_destination)

In [ ]:
os.listdir(new_destination)

### remove directory recursively

In [ ]:
shutil.rmtree(new_destination)

### copy a directory recursively

In [ ]:
import os
os.makedirs("some/deep/recursive/directory")

In [ ]:
destination = "destination_of_recursive_dir"
os.mkdir(destination)

if you repeat the above (the directory already exists), you'll get a `FileExistsError`:

In [ ]:
os.mkdir(destination)

**catch the `FileExistsError` error**

In [ ]:
import os
try:
    os.mkdir(destination)
except FileExistsError:  # catch this specific error
    pass  # do nothing

now, deep-copy the directory:

In [ ]:
shutil.copytree('some', destination)

now you realize, in the Python standard library, there are sometimes **very annoying limitations**. The code below will only work with Python 3.8 and onward:

In [ ]:
shutil.copytree('some', destination, dirs_exist_ok=False)

Of course, there is a workaround which works nicely from the beginning:

In [ ]:
from distutils.dir_util import copy_tree
copy_tree("some", destination)

**Google and StackOverflow are your friends. Don't hesistate to consult them for the (currently) best solution to your problem :)**

## Filename pattern search: `glob`

In [2]:
import glob

In [3]:
for filename in glob.glob("*.ipynb"):
    print(filename)

01_interaction_with_the_file_system.ipynb
03_open_and_close_files.ipynb
02_interaction_with_file_system_pathlib.ipynb


In [4]:
for filename in glob.glob("01_*.ipynb"):
    print(filename)

01_interaction_with_the_file_system.ipynb


In [7]:
for filename in glob.glob("0?_interaction_with*.ipynb"):
    print(filename)

01_interaction_with_the_file_system.ipynb
02_interaction_with_file_system_pathlib.ipynb
